In [4]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data
import copy
from tqdm.notebook import tqdm_notebook
import math
from skopt import gbrt_minimize
import warnings
warnings.filterwarnings('ignore')

device = torch.device("mps" if torch.has_mps else "cpu")
print(device)
from itertools import product
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

random_state = 1

mps


In [5]:
# read in data and split
X = torch.load('inputs/rfw_resnet50_face_embeddings.pt').cpu()
y = torch.load('inputs/rfw_resnet50_labels.pt').cpu()
df = pd.read_csv('inputs/rfw_resnet50_df.csv')
df['reference_ethnicity'] = df['reference_ethnicity'].str.lower()
ethnicity = df['reference_ethnicity']
df

,reference_identity,candidate_identity,reference_ethnicity,candidate_ethnicity,labels
0,m.0c7mh2,m.0c7mh2,african,African,1.0
1,m.0c7mh2,m.0c7mh2,african,African,1.0
2,m.026tq86,m.026tq86,african,African,1.0
3,m.026tq86,m.026tq86,african,African,1.0
4,m.02wz3nc,m.02wz3nc,african,African,1.0
...,...,...,...,...,...
29311,m.0402tg,m.01npnk3,caucasian,Caucasian,0.0
29312,m.05pbbnj,m.02rrb2n,caucasian,Caucasian,0.0
29313,m.09j6df,m.07kcsqd,african,African,0.0
29314,m.0fhrbz,m.025zgjt,african,African,0.0


In [6]:
train_split, test_split = train_test_split(np.arange(len(X)),test_size=0.2, random_state=random_state)
train_split, val_split = train_test_split(train_split,test_size=0.25, random_state=random_state)
X_train = X[train_split]
X_val = X[val_split]
X_test = X[test_split]
y_train = y[train_split]
y_val = y[val_split]
y_test = y[test_split]

ethnicity_train = ethnicity[train_split].values
ethnicity_train[ethnicity_train=='caucasian'] = 0
ethnicity_train[ethnicity_train=='african'] = 1
ethnicity_train = ethnicity_train.astype(int)

ethnicity_val = ethnicity[val_split].values
ethnicity_val[ethnicity_val=='caucasian'] = 0
ethnicity_val[ethnicity_val=='african'] = 1
ethnicity_val = ethnicity_val.astype(int)

ethnicity_test = ethnicity[test_split].values
ethnicity_test[ethnicity_test=='caucasian'] = 0
ethnicity_test[ethnicity_test=='african'] = 1
ethnicity_test = ethnicity_test.astype(int)

In [7]:
train_df = df.iloc[train_split]
val_df = df.iloc[val_split]
test_df = df.iloc[test_split]
test_df

,reference_identity,candidate_identity,reference_ethnicity,candidate_ethnicity,labels
18904,m.0cx09_,m.02q_nsj,caucasian,Caucasian,0.0
28620,m.080gs5b,m.027w0nt,african,African,0.0
11687,m.027q3qg,m.027q3qg,caucasian,Caucasian,1.0
27477,m.034s_j,m.03h3058,african,African,0.0
12283,m.05p7_tb,m.05p7_tb,caucasian,Caucasian,1.0
...,...,...,...,...,...
18134,m.0hzpkcs,m.067fmb,caucasian,Caucasian,0.0
3639,m.05lnvt,m.05lnvt,african,African,1.0
11091,m.01ndxh,m.01ndxh,caucasian,Caucasian,1.0
9369,m.0c2g8y,m.0c2g8y,caucasian,Caucasian,1.0


In [8]:
## train data
class TrainData(data.Dataset):
    
    def __init__(self, X_data, y_data,ethnicity):
        self.X_data = X_data
        self.y_data = y_data
        self.ethnicity = ethnicity    
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index], self.ethnicity[index]
        
    def __len__ (self):
        return len(self.X_data)


train_data = TrainData(torch.FloatTensor(X_train), 
                       torch.FloatTensor(y_train),
                       ethnicity_train)
test_data = TrainData(torch.FloatTensor(X_test),torch.FloatTensor(y_test),ethnicity_test)
val_data = TrainData(torch.FloatTensor(X_val), 
                       torch.FloatTensor(y_val),
                       ethnicity_val)
class BinaryClassification(nn.Module):
    def __init__(self):
        super(BinaryClassification, self).__init__()
        # Number of input features is 4096
        self.layer_1 = nn.Linear(4096, 1024) 
        self.layer_2 = nn.Linear(1024, 512)
        self.layer_out = nn.Linear(512, 1) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(1024)
        self.batchnorm2 = nn.BatchNorm1d(512)
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.layer_out(x)
        
        return x

def confusion_mat(y_pred, y_test):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    acc = (tn + tp)/(tn+tp+fn+fp)
    return tn, fp, fn, tp, acc
    


In [9]:
EPOCHS = 20
BATCH_SIZE = 64
LEARNING_RATE = 1e-5

model = torch.load('weights/rfw_resnet50_logistic_regression_face_matching_0.0_betaTEST.pt')
model.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

train_loader = data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = data.DataLoader(dataset=test_data, batch_size=BATCH_SIZE)
val_loader = data.DataLoader(dataset=val_data, batch_size=BATCH_SIZE)


In [10]:
def val_model(model, loader, criterion):
    """Validate model on loader with criterion function"""
    y_true, y_pred, y_prot = [], [], []
    model.eval()
    with torch.no_grad():
        for inputs, labels,protected in loader:
            inputs = inputs.to(device)
            y_true.append(labels)
            y_prot.append(protected)
            y_pred.append(torch.sigmoid(model(inputs)).cpu())
    y_true, y_pred, y_prot = torch.cat(y_true), torch.cat(y_pred), torch.cat(y_prot)
    return criterion(y_true, y_pred, y_prot)
def compute_bias(y_pred, y_true, prot, metric):
    """Compute bias on the dataset"""
    def zero_if_nan(data):
        """Zero if there is a nan"""
        return 0. if torch.isnan(data) else data

    gtpr_prot = zero_if_nan(y_pred[prot * y_true == 1].mean())
    gfpr_prot = zero_if_nan(y_pred[prot * (1-y_true) == 1].mean())
    mean_prot = zero_if_nan(y_pred[prot == 1].mean())

    gtpr_unprot = zero_if_nan(y_pred[(1-prot) * y_true == 1].mean())
    gfpr_unprot = zero_if_nan(y_pred[(1-prot) * (1-y_true) == 1].mean())
    mean_unprot = zero_if_nan(y_pred[(1-prot) == 1].mean())

    if metric == "spd":
        return mean_prot - mean_unprot
    elif metric == "aod":
        return 0.5 * ((gfpr_prot - gfpr_unprot) + (gtpr_prot - gtpr_unprot))
    elif metric == "eod":
        return gtpr_prot - gtpr_unprot
    elif 'false_diff':
        return (np.abs(gfpr_prot - gfpr_unprot) + np.abs(gtpr_unprot - gtpr_prot))

def compute_objective(performance, bias, epsilon=0.001, margin=0.0005):
    if abs(bias) <= (epsilon-margin):
        return performance
    else:
        return 0.0
def get_best_objective(y_true, y_pred, y_prot):
    """Find the threshold for the best objective"""
    num_samples = 5
    threshs = torch.linspace(0, 1, 501)
    best_obj, best_thresh = -math.inf, 0.
    for thresh in threshs:
        indices = np.random.choice(np.arange(y_pred.size()[0]), num_samples*y_pred.size()[0], replace=True).reshape(num_samples, y_pred.size()[0])
        objs = []
        for index in indices:
            y_pred_tmp = y_pred[index]
            y_true_tmp = y_true[index]
            y_prot_tmp = y_prot[index]
            perf = (torch.mean((y_pred_tmp > thresh)[y_true_tmp.type(torch.bool)].type(torch.float32)) + torch.mean((y_pred_tmp <= thresh)[~y_true_tmp.type(torch.bool)].type(torch.float32))) / 2
            bias = compute_bias((y_pred_tmp > thresh).float().cpu(), y_true_tmp.float().cpu(), y_prot_tmp.float().cpu(), 'false_diff')
            objs.append(compute_objective(perf, bias))
        obj = float(torch.tensor(objs).mean())
        if obj > best_obj:
            best_obj, best_thresh = obj, thresh

    return best_obj, best_thresh

In [11]:
# compute bias before applying random processing
y_true, y_pred, y_prot = [], [], []
model.eval()
with torch.no_grad():
    for inputs, labels,protected in test_loader:
        inputs = inputs.to(device)
        y_true.append(labels)
        y_prot.append(protected)
        y_pred.append(torch.sigmoid(model(inputs)).cpu())
y_true, y_pred, y_prot = torch.cat(y_true), torch.cat(y_pred), torch.cat(y_prot)

compute_bias((y_pred>0.5).float().cpu(), y_true, y_prot, 'false_diff')


tensor(0.0185)

In [12]:
num_iterations = 200
torch.manual_seed(random_state)
np.random.seed(random_state)
rand_result = [-np.inf, None, -1]

for iteration in tqdm_notebook(range(num_iterations),total=num_iterations):
    rand_model = copy.deepcopy(model)
    rand_model = rand_model.to(device)
    for param in rand_model.parameters():
        param.data = param.data * (torch.randn_like(param) * 0.1 + 1)

    rand_model.eval()
    best_obj, best_thresh = val_model(rand_model, val_loader, get_best_objective)

    if best_obj > rand_result[0]:

        del rand_result[1]
        rand_result = [best_obj, copy.deepcopy(rand_model.state_dict()), best_thresh]

# evaluate best random model
best_model = copy.deepcopy(model)
best_model.load_state_dict(rand_result[1])
best_model.to(device)
best_thresh = rand_result[2]

y_true, y_pred, y_prot = [], [], []
best_model.eval()
with torch.no_grad():
    for inputs, labels,protected in val_loader:
        inputs = inputs.to(device)
        y_true.append(labels)
        y_prot.append(protected)
        y_pred.append(torch.sigmoid(best_model(inputs)).cpu())
y_true, y_pred, y_prot = torch.cat(y_true), torch.cat(y_pred), torch.cat(y_prot)
compute_bias((y_pred>best_thresh).float().cpu(), y_true, y_prot, 'false_diff')
# torch.save(best_model.state_dict(), 'seed'+str(random_state)+'aod'+config['optimizer']+)

  0%|          | 0/200 [00:00<?, ?it/s]

tensor(0.)